In [0]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, TimestampType,DateType


In [0]:
from pyspark.sql.functions import col,lit,current_timestamp

In [0]:
cust_schema = StructType([ StructField("created_timestamp",TimestampType(),True),
                           StructField("customer_id",IntegerType(),True),
                           StructField("customer_name",StringType(),True),
                            StructField("date_of_birth",DateType(),True),
                            StructField("email",StringType(),True),
                            StructField("member_since",DateType(),True),
                            StructField("telephone",StringType(),True)])

In [0]:
df = (
       spark.readStream 
          .format("json") 
          .schema(cust_schema) 
          .load("/Volumes/gizmobox/landing/operational/customers_stream/")
    )
df.display(0)

In [0]:
df_new_columns = df.withColumn("filepath",col("_metadata.file_path")) \
                   .withColumn("ingestiondate",current_timestamp())
df_new_columns.display()

In [0]:
stream_query = (
                df_new_columns.writeStream 
                             .format("delta") 
                             .option("checkpointLocation", "/Volumes/gizmobox/landing/operational/customers_stream/_checkpointsstream")
                             .toTable("gizmobox.bronze.customers_stream")
                )

In [0]:
%sql
select * from gizmobox.bronze.customers_stream